# Fine-tuning led-large-book-summary on fine-tuning terms and conditions.

This notebook will go through data-processing and training of a the NLP model responsible for summarising terms and conditions, using the "led-large-book-summary" as a base model to be fine-tuned.

led-large-book-summary: https://huggingface.co/pszemraj/led-large-book-summary

This will be a very similar procedure to training on privacy policies, but with a dataset containing terms and conditons and their respective summaries.

**This model needs ~16GB of VRAM for training, running on any lower will give a CUDA out of memory error.**

# Pre-Processing
First, import and install required libraries

In [23]:
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
from datasets import load_dataset, DatasetDict
import os

# Allocate maximum CUDA memory reserve in an attempt to prevent CUDA out of memory errors
# Reserve is simply the reserved memory, not the in-use memory.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


First, the dataset needs to be converted from JSONL to a "dataset" object from the `datasets` library.

This library provides a `train_test_split` to split the dataset into a test set and training set.

I shuffle the dataset first with a fixed seed, so the results are always repoducible.

In [24]:
# Dataset files are stored in different locations depending on where the Notebook is ran
# Uncomment depending on location:

# Kaggle:
dataset_location = "/kaggle/input/terms-jsonl/Terms_dataset.jsonl"

# Google Colab / Running Locally:
# dataset_location = "dataset.json"

dataset = load_dataset("json", data_files=dataset_location,split='train')
dataset = dataset.shuffle(seed=2424)
dataset = dataset.train_test_split(test_size=0.1, shuffle=False) # disabling shuffling to shuffle with a fixed seed on previous line instead
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['summary', 'document'],
        num_rows: 258
    })
    test: Dataset({
        features: ['summary', 'document'],
        num_rows: 29
    })
})


The dataset has been split into two sets:
 - Train
 - Test

The train set will be used to train the model - this is the information the model will learn from.
The test set will be used to test the model after training, to see how it performs for some data **It has never seen**

The dataset has two features - the "document" which is a terms document and then "summary" is the respective summary of thwese terms

Next, a model needs to be selected to conduct transfer learning on.

There is a problem here, in that, the collected terms and conditions are **very long**.

Below, the length of the first item in the dataset is 81,271 characters whilst the largest is 771,229 characters.

In [25]:
length_of_first_item = len(dataset['train'][0]['document'])
print(f'The length of the first terms and conditions in the train dataset is {length_of_first_item} characters')

length_of_longest_document = len(max(dataset['train'], key=lambda x: len(x['document']))['document'])
print(f'Length of the longest terms and conditions in the train dataset is {length_of_longest_document} characters')

The length of the first terms and conditions in the train dataset is 81271 characters
Length of the longest terms and conditions in the train dataset is 771229 characters


This is problematic as NLP models have a **maximum token count** that it can handle, often much lower than the length of the collected documents, exacerbated by the fact summarisation models typically have lower maximum token counts.

This may end up affecting the accuracy of the model and it's ability to learn from the data, as if a portion of the document is cut off to stay within the maximum token count, the summary may not fully match the document.

Unfortunately, this is a limitation of NLP as a whole.

The maximum length of most of the most common summarization models is 1024 tokens, it's impossible to tell how many characters this is, but a rough heuristic is [1 token = 4 characters](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them) 

The model below thus supports around 4096 characters, which is clearly not good enough for the data.

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Commented out to save memory for training...

# bart_model_checkpoint_name = "facebook/bart-large-cnn"
# bart_tokenizer = AutoTokenizer.from_pretrained(bart_model_checkpoint_name)
# bart_max_length = bart_tokenizer.model_max_length
# bart_max_length, bart_max_length*4

Thus, it's required to look at a bigger model.

Below, "led-large-book-summary" has a model max length of 16384 tokens, applying the same heuristic as before, this is about 65536 words.

This model was trained on the [BookSum dataset](https://arxiv.org/abs/2105.08209) which contains "plays, short stories, and novels" with expired copyright, and the aim of the training was to produce valuable summaries of the given documents.

I will conduct transfer learning to remove the head of this model which is specifically focused on books, and train on the new task of summarising **privacy policies**

In [27]:
model_checkpoint_name = "pszemraj/led-large-book-summary"
tokeniser = AutoTokenizer.from_pretrained(model_checkpoint_name)
tokeniser.model_max_length, tokeniser.model_max_length * 4

(16384, 65536)

The above has taken the "tokeniser" from the model.

Transformer models only take numerical inputs, thus, it's essential to get a numerical representation of the input.

The tokeniser is responsible for turning sentences into a series of numbers, called "tokens".

Tokenisers also use special characters, often to indicate the start and end of sentences or words.

All this information helps a NLP neural network form an understanding of the input.

The below output shows the tokens used to encode the given test string, with `<s>` being used to indicate the start of a sequence, and each subsequent token beginning with `Ġ`, indicating the start of a new token.

Note that `tokeniser` was split into two tokens `token` and `iser`, this is important for tokenisers to be able to re-use tokens wherever possible - `iser` can be used as the suffix for many different words.

An attention mask can tell the tokenizer not to pay attention to certain tokens.

In [28]:
test_string = tokeniser("This is a test string to test out the tokeniser")
print(test_string)
tokeniser.convert_ids_to_tokens(test_string.input_ids)

{'input_ids': [0, 713, 16, 10, 1296, 6755, 7, 1296, 66, 5, 19233, 5999, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


['<s>',
 'This',
 'Ġis',
 'Ġa',
 'Ġtest',
 'Ġstring',
 'Ġto',
 'Ġtest',
 'Ġout',
 'Ġthe',
 'Ġtoken',
 'iser',
 '</s>']

Next, recall that the maximum token size for the model was 16384, or about 65536 words.

The majority of the scraped documents still exceed this amount, unfortunately, since no bigger summarisation model exists, the input text will need to be truncated down to the maximum token size, which **will have an impact on the model's ability to learn** but there is no other option.

Furthermore, the tokenizer will output may different outputs, one of which being `input_ids`, the numerical IDs of the tokens in the tokenised text.

The `input_ids` will need to be assigned to the `labels` key of the tokenised input document. This prepares the tokenised input for a task where the model needs to predict the summary, which is how the model will learn.

Below defines and runs a function which will truncate the input document, **but not the output** and assign the labels and input id's as necessary

In [29]:
def truncate_input_tokens(input):
    truncated_input = tokeniser(
        input["document"],
        max_length = 16384,
        truncation = True
    )
    labels = tokeniser( # don't truncate output
        input["summary"],
        truncation = False,
    )
    truncated_input["labels"] = labels["input_ids"]
    return truncated_input

# By passing `batched = true` into the map function, more than one item is applied to the function at a time.
tokenised_dataset = dataset.map(truncate_input_tokens, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Below now looks at the new dataset.

We can see new features `input_ids`, `attention_mask` and `labels`

The lengths of these inputs are also visible, both at 16384 (the maximum) tokens for the first input, and the biggest.

The smallest input however, is just 694 tokens, suggesting some documents stayed well under the maximum.

In [30]:
print(tokenised_dataset)

length_of_first_item = len(tokenised_dataset['train'][0]['input_ids'])
print(f'The length of the first terms in the train dataset is {length_of_first_item} tokens')

length_of_longest_document = len(max(tokenised_dataset['train'], key=lambda x: len(x['input_ids']))['input_ids'])
print(f'Length of the longest terms in the train dataset is {length_of_longest_document} tokens')

length_of_longest_document = len(min(tokenised_dataset['train'], key=lambda x: len(x['input_ids']))['input_ids'])
print(f'Length of the smallest terms in the train dataset is {length_of_longest_document} tokens')

DatasetDict({
    train: Dataset({
        features: ['summary', 'document', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 258
    })
    test: Dataset({
        features: ['summary', 'document', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 29
    })
})
The length of the first terms in the train dataset is 16384 tokens
Length of the longest terms in the train dataset is 16384 tokens
Length of the smallest terms in the train dataset is 694 tokens


# Defining Evaluation Techniques

In Evaluation of summarisation tasks in NLP, the ROUGE metric is most often used.

To evaluate the model, I will use the `Hugging Face evaluate` library, as well as the `rouge-score` library to produce the rouge score for the model's summaries.

In [31]:
!pip install evaluate
!pip install rouge_score

import evaluate

rouge = evaluate.load("rouge")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


To calculate the ROUGE metric on predictions, we need the **decoded** predictions and the **decoded** labels, as they are currently *tokenised* (labels simply refers to the expected values).

Thus, we need to decode the input tokens back into their English representations.

The `batch-decode` function does this, with the parameter `skip_special_tokens` specifying to not handle any special tokens such as `<s>` indicating the start of a sequence. 

The function below will decode and find the ROUGE score.

In [32]:
import numpy as np

def decode_and_find_rouge(result):
    predictions,labels = result # extract predictions and labels from the passed in result.
    
    # Decode by converting tokenised inputs back into English representations
    decoded_predictions = tokeniser.batch_decode(predictions, skip_special_tokens=True) 
    decoded_labels = tokeniser.batch_decode(labels, skip_special_tokens = True)

    rouge_score = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

    num_of_predictions = [np.count_nonzero(pred != tokeniser.pad_token_id) for pred in predictions]
    rouge_score["gen_len"] = np.mean(num_of_predictions)

    return {k: round(v, 4) for k, v in rouge_score.items()}

# Training the Model

First, the model hyper-paramters need to be defined.

This is a very expensive model on memory and it is essential to avoid exceeding the GPU's VRAM, thus the hyper-parameters are configured as such:

1) The batch size will be set to just 1 to reduce the amount of data stored in memory.

2) 16-bit floating point precision will be used, rather than 32-bit.

3) The optimiser "adafactor" is used over the industry standard of "Adam". As a result, the model converges slower but also uses less memory.

4) "Gradient Checkpointing" is used, instructing the model to forget the majority of forward-pass activations and instead recompute them on demand during the backward pass, saving only the "most important" activations. Significantly slower training time, but also saves a lot of VRAM.

5) "Gradient Accumulation" is used simulate a "larger" effective batch size. Instead of updating the model's parameters after processing eachbatch, the gradients are accumulated over 2 batches before performing a single update.

In [33]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_name)

# Creating a data collator, which will form the batches which will be fed to the model.
# It will also conduct padding if necessary to get all inputs of equal length.
data_collator = DataCollatorForSeq2Seq(tokenizer=tokeniser, model=model_checkpoint_name)

models_arguments = Seq2SeqTrainingArguments(
    output_dir="TOS_4_Epoch_Model",
    evaluation_strategy="epoch", # Run evaluation function on each epoch
    learning_rate=2e-5, # learning rate hyperparameter set to 0.00002
    per_device_train_batch_size= 1, # split into batches of 1 for training
    per_device_eval_batch_size=1, # split to batches of 1 for evaluation
    weight_decay=0.01, # Utilises L2 regularization in an attempt to prevent overfitting
    save_total_limit=3, # save 3 checkpoints only and delete older checkpoints (Kept using all RAM without)
    num_train_epochs=4, # train for 3 epochs
    predict_with_generate=True, # Generate summaries for each input ; essential for summarisation tasks
    fp16=True, # use 16 bit floating point - reduced memory usage
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adafactor"
)

# Collect the previously defined trainer parameters, such as the evaluation technique, tokeniser and datasets.
trainer = Seq2SeqTrainer(
    model=model,
    args=models_arguments,
    train_dataset=tokenised_dataset["train"],
    eval_dataset=tokenised_dataset["test"],
    tokenizer=tokeniser,
    data_collator=data_collator,
    compute_metrics=decode_and_find_rouge,
)

Finally, the model can be ran using the `train()` function on the trainer.

After training, the model will be saved in a file called "trained_model".
 - The size of the model is ~1.7GB

In [34]:
import os

# Uncomment below line if using Kaggle.
os.environ["WANDB_DISABLED"] = "true"

trainer.train()
trainer.save_model("TOS_4_Epoch_Model")

You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.693679,0.480600,0.286600,0.250900,0.250300,297.379300
2,No log,0.593070,0.458700,0.298000,0.271000,0.270800,156.689700
3,No log,0.552057,0.447100,0.275600,0.277100,0.275600,158.275900
4,0.642000,0.547224,0.440600,0.262800,0.255000,0.254500,139.655200


# Evaluating the model

First, we can observe the output of the model on some examples, to see how the model is handling terms and conditions

In [38]:
from transformers import pipeline, AutoTokenizer, AutoModel,LEDTokenizer, LEDForConditionalGeneration

trained_tokeniser = LEDTokenizer.from_pretrained("/kaggle/working/TOS_4_Epoch_Model")

trained_model = LEDForConditionalGeneration.from_pretrained("/kaggle/working/TOS_4_Epoch_Model")

summarizer = pipeline("summarization", model=trained_model,tokenizer=trained_tokeniser)


example = dataset["test"]["document"][0]

tokenised_example = trained_tokeniser(example,return_tensors="pt",truncation=True).input_ids


outputs = trained_model.generate(tokenised_example)
trained_tokeniser.decode(outputs[0], skip_special_tokens=True)

"The service is provided 'as is' and to be used at the users' sole risk. The service provider makes no warranty regarding uninterrupted, timely, secure or error-free service. Failure to enforce any provision of the Terms of Service does not constitute a waiver of such provision. This service assumes no liability for any losses or damages resulting from any matter relating to the service. You agree to defend, indemnify, and hold the service harmless in case of a claim related to your use of the service"

In [39]:
second_example = dataset["test"]["document"][1]

tokenised_example_2 = trained_tokeniser(second_example,return_tensors="pt",truncation=True).input_ids

outputs_2 = trained_model.generate(tokenised_example_2)
trained_tokeniser.decode(outputs_2[0], skip_special_tokens=True)

"The service is provided 'as is' and to be used at the users' sole risk. The service provider makes no warranty regarding uninterrupted, timely, secure or error-free service. This service assumes no liability for any losses or damages resulting from any matter relating to the service. You waive your right to a class action.. Your account can be deleted without prior notice and without a reason.  Terms may be changed any time at their discretion, without notice to you. Instead of asking directly, this Service will assume your consent merely from your usage.. There is a date of the last update of the agreements (January 2020)"

In [40]:
lorem_example = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum"

tokenised_example_lorem = trained_tokeniser(lorem_example,return_tensors="pt",truncation=True).input_ids

outputs_lorem = trained_model.generate(tokenised_example_lorem)
trained_tokeniser.decode(outputs_lorem[0], skip_special_tokens=True)

"The service is provided 'as is' and to be used at your sole risk. You are responsible for any risks, damages, or losses that may incur by downloading materials. This service does not guarantee that it or the products obtained through it meet your expectations or requirements."

On the surface these summaries look sufficient.

We can look at the ROUGE scores of a test set example:

In [41]:
example_document = dataset["test"]["document"][5]
ground_truth_summary = dataset["test"]["summary"][5]

tokenised_document = trained_tokeniser(example_document,return_tensors="pt",truncation=True).input_ids

tokenised_ground_truth = trained_tokeniser(ground_truth_summary,return_tensors="pt",truncation=True).input_ids

output_summary = trained_model.generate(tokenised_document)

decoded_summary = trained_tokeniser.decode(output_summary[0], skip_special_tokens=True)

# Rouge metric expects array.
truth = [ground_truth_summary]
summary = [decoded_summary]

rouge_score = rouge.compute(predictions=truth, references=summary)

print(rouge_score)

{'rouge1': 0.49402390438247007, 'rouge2': 0.24096385542168675, 'rougeL': 0.26294820717131473, 'rougeLsum': 0.26294820717131473}


We can now take the ROUGE score across the entire test set

This will take a while, due to needing to create summaries for the entire test set.

In [ ]:
summaries = []
truths = []
count = 1

for item in dataset["test"]:
    print(f'Processing document {count} of {len(dataset["test"])}')
    document = item["document"]
    ground_truth_summary = item["summary"]
    
    tokenised_document = trained_tokeniser(document,return_tensors="pt",truncation=True).input_ids
    generated_summary = trained_model.generate(tokenised_document)
    decoded_summary = trained_tokeniser.decode(generated_summary[0], skip_special_tokens=True)
    
    summaries.append(decoded_summary)
    truths.append(ground_truth_summary)
    count += 1

In [44]:
rouge_score = rouge.compute(predictions=summaries,references=truths)
print(rouge_score)

{'rouge1': 0.45041833108246265, 'rouge2': 0.2748145311460465, 'rougeL': 0.2588148235927864, 'rougeLsum': 0.2582850488220999}
